---
title: 'Tutorial: How to fine tune a Cellpose model with the Active Learning plugin for Napari'
author: Fernando Cervantes
categories:
  - Cellpose
  - Transfer learning
  - napari
description: Tutorial for fine-tuning the "nuclei" model from `cellpose` to new data.

jupyter: python3
---

# Install `napari` and the Active Learning plugin

### Install `napari`

Follow the instructions to install `napari` from its [official website](https://napari.org/stable/tutorials/fundamentals/installation.html#napari-installation).

### Install the `napari-activelearning` plugin using `pip`

::: {.callout-important}
If you installed `napari` using conda, activate that same environment before installing the _Active Learning_ plugin.
:::

Install the plugin adding `[cellpose]` to the command so all dependencies required for this tutorial are available.

``` {.bash}
python -m pip install "napari-activelearning[cellpose]"
```

In [ ]:
import napari
import napari_activelearning as al
import numpy as np

### Launch `napari`

::: {.callout-caution}
The first time `napari` is launched it can take more than $30$ seconds to open, so please be patient!
:::

In [ ]:
viewer = napari.Viewer()

# Image groups management

## Load a sample image

You can use the cells 3D image sample from napari's built-in samples.

```
File > Open Sample > napari builtins > Cells (3D+2Ch)
```

In [ ]:
viewer.window._qt_window.showMaximized()
_ = viewer.open_sample(plugin="napari", sample="cells3d")

## Add the _Image Groups Manager_ widget to napari's window

The _Image groups manager_ can be found under the _Active Learning_ plugin in napari's plugins menu.

```
Plugins > Active Learning > Image groups manager
```

::: {.callout-note title="About the _Image groups manager_" collapse="true"}
The _Image groups manager_ widget is used to create _groups_ of layers and their specific use within the `active-learning` plugin.
It allows to specify properties and metadata of such layers (e.g. inputs paths, output directory, axes order, etc.) that are used by the _Acquisition Function Configuration_ widget.

This widget allows to relate different layers using a single hierarchical structure (_image group_).
Such hierarchical structure is implemented as follows:

* *Image groups*. Used to contain different modes of data from a single object, such as image pixels, labels, masks, etc.

    + *Layers groups*. These group multiple layers of the same mode together, e.g. different channel layers of the same image.

        - *Layer channels*. This is directly related to elements in `napari`'s _layer list_. Moreover, _layer channels_ allow to store additional information or _metadata_ useful for the inference and fine-tuning processes.
:::

In [ ]:
image_groups_mgr, acquisition_fun_cfg, label_groups_mgr = al.get_active_learning_widget()

In [ ]:
image_groups_mgr_dw = viewer.window.add_dock_widget(image_groups_mgr)

## Create an _Image Group_ containing _nuclei_ and _membrane_ layers

1. Select the _nuclei_ and _membrane_ layer.

2. Click the _New Image Group_ button on the _Image Groups Manager_ widget.

In [ ]:
viewer.layers.selection.clear()
viewer.layers.selection.add(viewer.layers["nuclei"])
viewer.layers.selection.add(viewer.layers["membrane"])

In [ ]:
image_groups_mgr.create_group()

## Edit the image group properties

1. Select the "images" layers group_ that has been created under the new image group.

2. Click the _Edit group properties_ checkbox.

3. Make sure that _Axes order_ is "CZYX", otherwise, you can edit it and press _Enter_ to update the axes names.


In [ ]:
focus_widget = image_groups_mgr.image_groups_editor.layout().itemAt(0).widget()
focus_widget.setChecked(True)

image_groups_mgr._active_image_group.child(0).setSelected(True)
image_groups_mgr.image_groups_editor.edit_axes_le.setText("CZYX")
image_groups_mgr.image_groups_editor.update_source_axes()

# Segmentation on image groups

## Add the _Acquisition function configuration_ widget to napari's window

The _Acquisition function configuration_ is under the _Active Learning_ plugin in napari's plugins menu.

```
Plugins > Active Learning > Acquisition function configuration
```

::: {.callout-tip}
All _Active Learning_ widgets can be _un-docked_ from their current place and _re-docked_ into other more convenient location within `napari`'s window, or even as tabs, as illustrated in this tutorial.
:::

::: {.callout-note title="About the _Acquisition function configuration_" collapse="true"}
The _Acquisition function configuration_ widget comprises the processes of _inference_ and _fine tunning_ of deep learning models.

This tool allows to define the sampling configuration used to retrieve patches from the _input_ image at random.
Such configuration involves the size on each dimension of the patches, the maximum number of patches that can be sampled from each image, and the order of the axes that are passed to the model's inference function.
Parameters for such model can also be configured within this widget.

The fine-tuning process can also be configured and launched from within this widget.
Finally, this widget executes the inference process on the same sampled patches to compare the _base_ and _fine-tuned_ model performance visually.
:::

In [ ]:
acquisition_fun_cfg_dw = viewer.window.add_dock_widget(acquisition_fun_cfg)
viewer.window._qt_window.tabifyDockWidget(image_groups_mgr_dw, acquisition_fun_cfg_dw)

image_groups_mgr_dw.setWindowTitle("Image groups manager")
acquisition_fun_cfg_dw.setWindowTitle("Acquisition function configuration")

In [ ]:
acquisition_fun_cfg_dw.raise_()

## Set the size of the sampling patch

1. Click the "Edit patch size" checkbox
2. Change the patch size of "X" and "Y" to 256, and the "Z" axis to 1.

In [ ]:
focus_widget = acquisition_fun_cfg.layout().itemAt(0).widget()
focus_widget.setChecked(True)

acquisition_fun_cfg.patch_sizes_mspn.sizes = {"Z": 1, "Y": 256, "X": 256}

## Define the maximum number of samples to extract

:::{.callout-note}
This tells the Active Learning plugin to process at most _four_ samples at random from the whole image.

Because the image is of size $256\times256$ pixels, four whole slices will be sampled at random for segmentation.
:::

In [ ]:
acquisition_fun_cfg.max_samples_spn.setValue(4)

## Configure the segmentation method

1. Use the "Model" dropdown list to select the `cellpose` method

2. Click the "Advanced segmentation parameters" checkbox

3. Change the second channel to $1$ (the right spin box in the "channels" row)

::: {.callout-note}
This tells `cellpose` to segment the first channel ($0$) and use the second channel ($1$) as help channel.
:::

4. Choose the "nuclei" model from the dropdown list


In [ ]:
acquisition_fun_cfg.methods_cmb.setCurrentIndex(2)
acquisition_fun_cfg.tunable_segmentation_method.advanced_segmentation_options_chk.setChecked(True)
acquisition_fun_cfg.tunable_segmentation_method._segmentation_parameters.channels.value = (0, 1)
acquisition_fun_cfg.tunable_segmentation_method._segmentation_parameters.model_type.value = "nuclei"

## Execute the segmentation method on all image groups

- Click the "Run on all image groups" button

::: {.callout-note}
To execute the segmentation only on specific image groups, select the desired image groups in the _Image groups manager_ widget and use the "Run on selected image groups" button instead.
:::

In [ ]:
_ = acquisition_fun_cfg.compute_acquisition_layers(run_all=True)

## Inspect the segmentation layer

- We will only see four segmented slices from the whole image. This is because we set the "Maximum samples" parameter to $4$.

::: {.callout-note}
Because the input image is 3D, you may have to slide the "Z" index at the bottom of napari's window to look at the samples that were segmented.
:::

::: {.callout-note title="About the newly added layers" collapse="true"}
Three new layers were added to the `napari`'s layers list after executing the segmentation pipeline.

The "images acquisition function" presents a map of the confidence prediction made by the chosen model for each sampled patch.
Lighter intensity correspond to regions where the annotator should pay attention when correcting the output labels, since those are low-confidence predictions.
On the other hand, darker intensity corresponds to high-confidence predictions, and correcting these labels would not have much impact in the performance of the fine-tuned model as correcting low-confidence (lighter intensity) regions.

The "images sampled positions" show a low-resolution map of the regions that were sampled during the segmentation process.

The "images segmentation" layer are the output labels predicted by the model.

Note that the name of those three generated layers contain the name of the _image group_ from where those patches were sampled, in this case the "images" _image group_.
:::

In [ ]:
label_groups_mgr.focus_region(
  label_groups_mgr.labels_group_root.child(0).child(0)
)

# Segment masked regions only

## Create a mask to restrict the sampling space

1. Switch to the _Image groups manager_ tab

2. Click the "Edit mask properties" checkbox

3. Set the mask scale to $256$ for the "X" and "Y" axes, and a scale of $1$ for the "Z" axis

4. Click the "Create mask" button

::: {.callout-note}
This creates a low-resolution mask where each of its pixels corresponds to a $256\times256$ pixels region in the input image.
Because the mask is low-resolution, it uses less space in memory and disk.
The new mask will appear as "images mask (1)" since there already exists one mask that was generated automatically when running the segmentation method in step **3.5**.
:::

In [ ]:
image_groups_mgr_dw.raise_()

focus_widget = image_groups_mgr.mask_generator.layout().itemAt(0).widget()
focus_widget.setChecked(True)

image_groups_mgr.mask_generator.patch_sizes_mspn.sizes = {"Z": 1, "Y": 256, "X": 256}
image_groups_mgr.mask_generator.generate_mask_layer()

## Specify the samplable regions

1. Make sure the newly created layer "images mask (1)" is selected.

2. Activate the _fill bucket_ tool (or press *4* or *F* keys in the keyboard as shortcuts).

3. Click the image to draw the mask on the current slice.

4. Move the slider at the bottom of napari's window to navigate between slices in the "Z" axis. Select slice $28$ and draw on it as in step $3$. Repeat with slices $29$ and $30$.

In [ ]:
viewer.camera.center = (27, 128, 128)
viewer.dims.current_step = (27, 128, 128)

viewer.layers["images mask (1)"].data[27:31, 0, 0] = 1
viewer.layers["images mask (1)"].refresh()

## Execute the segmentation process on the masked regions

- Return to the _Acquisition function configuration_ tab and run the segmentation process again (clicking "Run on all image groups" button).

In [ ]:
acquisition_fun_cfg_dw.raise_()

In [ ]:
_ = acquisition_fun_cfg.compute_acquisition_layers(run_all=True)

## Inspect the results from the segmentation applied only to masked regions

- All slices between $27$ and $30$ were picked and segmented thanks to the sampling mask!

::: {.callout-note}
At this point, multiple layers have been added to `napari`'s layers list and it could start to look clutered.

The following layers will be used in the remainder of this tutorial: "membrane", "nuclei", "images mask (1)", and "images segmentation [1]".

The rest of the layers can be safely removed from the list (i.e. "images acquisition function [1]", "images acquisition function", "images mask", and "images segmentation").
:::

In [ ]:
viewer.layers.selection.clear()
for layer in viewer.layers:
    if layer.name in ("images mask", "images segmentation", "images acquisition function", "images acquisition function [1]"):
        viewer.layers.selection.add(layer)

In [ ]:
removable_layers = list(viewer.layers.selection)
for layer in removable_layers:
    viewer.layers.remove(layer)

# Fine tune the segmentation model

## Add the _Label groups manager_ widget to napari's window

You can find the _Label groups manager_ under the _Active Learning_ plugin in napari's plugins menu.

```
Plugins > Active Learning > Label groups manager
```

::: {.callout-note title="About the _Label groups manager_" collapse="true"}
The _Label groups manager_ widget contains the coordinates of the sampled patches acquired by the _Acquisition function configuration_ widget.

This widget allows to navigate between patches without serching them in the complete extent of their corresponding image.
It additionally permits to edit the pixel data of the selected patch's _labels_ in order to correct the inference output in a _human-in-the-loop_ approach.
:::

In [ ]:
label_groups_mgr_dw = viewer.window.add_dock_widget(label_groups_mgr)
viewer.window._qt_window.tabifyDockWidget(acquisition_fun_cfg_dw, label_groups_mgr_dw)

label_groups_mgr_dw.setWindowTitle("Label groups manager")

In [ ]:
label_groups_mgr_dw.raise_()

## Navigate between patches with the _Label groups manager_

1. Select the last _labels group_ "Labels of groups: images" added to the manager. If you have not removed any layers from `napari`'s layers list, it should be the second group, otherwise, it will be the only group available.

2. Select the first _label_ in such group.

3. Use the navigation buttons to move between labels.

::: {.callout-caution}
Selecting a _label_ element in the _label groups manager_ will toggle all layers visible.
You can hide again any layer that blocks the visibility of objects of interest.
:::

In [ ]:
label_groups_mgr.labels_table_tw.expandItem(label_groups_mgr.labels_group_root.child(0))
label_groups_mgr.labels_table_tw.setCurrentItem(label_groups_mgr.labels_group_root.child(0).child(0), 0)
label_groups_mgr.focus_region(
  label_groups_mgr.labels_group_root.child(0).child(0),
  edit_focused_label=True
)

## Use napari's layer controls to make changes on the objects of the current patch

1. Select the _label_ associated to the patch extracted at slice $27$. That label can be found by looking for the coordinate $(27, 0, 0)$ in the "Sampling top-left" column of the _label groups manager_ table.

::: {.callout-note}
This creates a new editable "Labels edit" layer with a copy of the selected patch.
:::

2. Make sure the "Labels edit" layer is selected.

3. Edit the labels on the current patch with `napari`'s annotation tools.
 
::: {.callout-tip title="How to use the annotation tools" collapse="true"}

1. Select the _pick mode_ tool (click the dropper icon or press key _5_ or _L_ on the keyboard) and click on an object in the current patch to get its label index.

2. Use the _paint brush_ (click the brush icon or press key _2_ or _P_ on the keyboard) and add pixels missed by the model.

3. Remove extra pixels with the _label eraser_ tool (click the eraser icon or press key _1_ or _E_ on the keyboard).

4. If an object was not segmented at all, press the _M_ key on the keyboard to get the next _unused_ label index, and use the _paint brush_ as in step $2$ to cover the pixels of that object.

For a more complete guide on annotation with `napari` follow [this tutorial](https://napari.org/stable/howtos/layers/labels.html#creating-deleting-merging-and-splitting-connected-components).

:::

4. Click the "Commit changes" button when finished editing.

In [ ]:
label_groups_mgr.labels_table_tw.setCurrentItem(label_groups_mgr.labels_group_root.child(0).child(3), 0)
label_groups_mgr.focus_region(
  label_groups_mgr.labels_group_root.child(0).child(3),
  edit_focused_label=True
)

## Edit labels on other slices

- Continue editing the labels of the other patches until these are closer to what the model is expected to predict.

::: {.callout-important}
The `cellpose` model used in this tutorial does not work with sparse labeling, so its important that all the objects that are of interest should be labeled.

Additionally, the performance of the _fine-tuned_ model will depend highly on how precise the objects were labeled.
:::

## Select the _layer group_ that will be used as labels for _fine-tuning_

1. Go to the _image groups manager_ widget and select the "segmentation (1)" _layers group_.

::: {.callout-tip}
The "Group name" column in the groups table can be resized to show the complete names.
:::

2. Open the _Edit group properties_ view (clicking on the checkbox) and tick the "Use as labels" checkbox.

::: {.callout-note}
Any _layers group_ can be used as labels for the _fine-tuning_ process, just make sure the layer type is appropriate for the training workflow of the model to fine-tune.

In this tutorial, the labels must be of `napari.layers.Labels` type.
:::

In [ ]:
image_groups_mgr_dw.raise_()

focus_widget = image_groups_mgr.mask_generator.layout().itemAt(0).widget()
focus_widget.setChecked(True)

image_groups_mgr.image_groups_tw.clearSelection()
image_groups_mgr.image_groups_tw.setColumnWidth(0, 200)

vertical_scroll_bar = image_groups_mgr.image_groups_tw.verticalScrollBar()
vertical_scroll_bar.setValue(vertical_scroll_bar.maximum())

image_groups_mgr.groups_root.child(0).child(2).setSelected(True)

image_groups_mgr.image_groups_editor.use_as_labels_chk.setChecked(True)

## Setup fine tuning configuration

1. Go to the _Acquisition function configuration_ widget and click the "Advanced fine tuning parameters" checkbox

2. Change the "save path" to a location where you want to store the fine tuned model

In [ ]:
acquisition_fun_cfg_dw.raise_()

In [ ]:
acquisition_fun_cfg.tunable_segmentation_method.advanced_finetuning_options_chk.setChecked(True)
acquisition_fun_cfg.tunable_segmentation_method._finetuning_parameters.save_path.value = "C:/Users/Public/Documents/models"

## Setup fine tuning configuration

4. Change the "model name" to "nuclei_ft"

::: {.callout-tip}
Scroll the _Advanced fine tuning parameters_ widget down to show more parameters.
:::

5. Set the "batch size" to $3$

6. Change the "learning rate" to $0.0001$

::: {.callout-tip}
You can modify other parameters for the training process here, such as the number of training epochs.
:::

In [ ]:
vertical_scroll_bar = acquisition_fun_cfg.tunable_segmentation_method._finetuning_parameters_scr.verticalScrollBar()
vertical_scroll_bar.setValue(vertical_scroll_bar.maximum())
 
acquisdition_fun_cfg.tunable_segmentation_method._finetuning_parameters.model_name.value = "nuclei_ft"
acquisition_fun_cfg.tunable_segmentation_method._finetuning_parameters.batch_size.value = 3
acquisition_fun_cfg.tunable_segmentation_method._finetuning_parameters.learning_rate.value = 0.0001

## Execute the fine tuning process

::: {.callout-caution title="Double check the fine-tuning parameters!" collapse="true"}
Because _spin boxes_ in this widget can be modified by _hovering_ and _scrolling_ with the mouse, it is possible to change their values when scrolling down the parameters window.

Make sure that the parameters are the same as the shown in the following screenshot, particularly the size of the crops used for _data augmentation_ (_bsize_), and the number of images per epoch (_nimg per epoch_).

![](finetuning_parameter_values.png)
:::

- Click the "Fine tune model" button to run the training process.

::: {.callout-caution}
Depending on your computer resources (RAM, CPU), this process might take some minutes to complete. If you have a dedicated GPU device, this can take a couple of seconds instead.
:::

In [ ]:
_ = acquisition_fun_cfg.fine_tune()

## Review the fine tuned segmentation

::: {.callout-tip}
Use the *opacity* slider to compare how the fine tuned model segments the same objects that were labeled for training.
:::

# Use the fine tuned model for inference

## Create a mask to apply the fine tuned model for inference

1. Follow the steps from **4.1** to create a mask, now covering slices $31$ to $34$.

2. Scroll down the _Image groups manager_ table to show the newest mask layer group ("mask (1)").

3. Select the "mask (1)" layer group

4. Open the _Edit group properties_ (by clicking its checkbox) and make sure that "Use as sampling mask" checkbox is checked.

6. Go to the _Acquisition function configuration_ widget again and click "Run on all image groups" button.

In [ ]:
image_groups_mgr_dw.raise_()

focus_widget = image_groups_mgr.mask_generator.layout().itemAt(0).widget()
focus_widget.setChecked(True)

image_groups_mgr.mask_generator.patch_sizes_mspn.sizes = {"Z": 1, "Y": 256, "X": 256}
image_groups_mgr.mask_generator.generate_mask_layer()

viewer.camera.center = (31, 128, 128)
viewer.dims.current_step = (31, 128, 128)

viewer.layers["images mask"].data[31:35, 0, 0] = 1
viewer.layers["images mask"].refresh()

image_groups_mgr.image_groups_tw.clearSelection()

image_groups_mgr.image_groups_tw.setColumnWidth(0, 200)

vertical_scroll_bar = image_groups_mgr.image_groups_tw.verticalScrollBar()
vertical_scroll_bar.setValue(vertical_scroll_bar.maximum())

image_groups_mgr.groups_root.child(0).child(image_groups_mgr.groups_root.child(0).childCount() - 1).setSelected(True)

## Inspect the output of the fine tuned model

- Now you have a fine tuned model for nuclei segmentation that has been adapted to this image!

::: {.callout-important}
The fine tuned model can be used in *Cellpose*'s GUI software, their python API package, or any other package that supports pretrained `cellpose` models.
:::

In [ ]:
_ = acquisition_fun_cfg.compute_acquisition_layers(run_all=True)

viewer.camera.center = (33, 128, 128)
viewer.dims.current_step = (33, 128, 128)

## Segment the newly masked region with the base model for comaprison

1. Choose the "nuclei" model again from the dropdown list in the _Advanced segmentation parameters_ section

2. Click the "Run on all image groups" button again

::: {.callout-note}
This will execute the segmentation process with the non-fine-tuned "nuclei" model on the same sampling positions from the last run.
:::

In [ ]:
acquisition_fun_cfg_dw.raise_()
 
vertical_scroll_bar = acquisition_fun_cfg.tunable_segmentation_method._segmentation_parameters_scr.verticalScrollBar()
vertical_scroll_bar.setValue(vertical_scroll_bar.maximum())

acquisition_fun_cfg.tunable_segmentation_method._segmentation_parameters.model_type.value = "nuclei"

In [ ]:
_ = acquisition_fun_cfg.compute_acquisition_layers(run_all=True)

## Compare the results of both models

- Remove all the layers except the "nuclei" and "membrane" layers, and the "images segmentation" and "images segmentation [2]" layers, that are the segmentation outputs from the fine-tuned model and base model, respectively.

::: {.callout-tip}
Click the **eye** icon in the "layer list" to hide/show layers instead of removing the layers.
:::

In [ ]:
viewer.layers.selection.clear()
for layer in viewer.layers:
    if layer.name not in ("images segmentation [2]", "images segmentation", "nuclei", "membrane"):
        viewer.layers.selection.add(layer)

In [ ]:
removable_layers = list(viewer.layers.selection)
for layer in removable_layers:
    viewer.layers.remove(layer)